# PointNet

Implemtation of the pointnet classifier model on the ModelNet10 dataset.

Original paper :- https://arxiv.org/abs/1612.00593

We have extended the dataset by adding certain random 90 degree rotations to the dataset and sampled 2048 points from each mesh. It can be found in `data/ModelNet10`

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Flatten, GlobalMaxPooling1D, Reshape, Dot, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.regularizers import Regularizer
from tensorflow.keras.initializers import Constant

from tqdm.notebook import tqdm
import numpy as np
from sys import platform
import os
import k3d

## Set Backend Device

In [2]:
if tf.test.gpu_device_name():
    print(f'Default GPU Device: {tf.test.gpu_device_name()}')

# Set usage to float32 to reduce memory usage
tf.keras.backend.set_floatx('float32')

Default GPU Device: /device:GPU:0


2023-09-06 02:04:16.335338: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2023-09-06 02:04:16.335351: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-09-06 02:04:16.335360: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-09-06 02:04:16.335389: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-09-06 02:04:16.335403: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-09-06 02:04:16.336274: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been buil

In [3]:
BATCH_SIZE = 24
EPOCHS = 6

In [4]:
train_pcs = np.load("./data/ModelNet10/train_pc.npz")
test_pcs = np.load("./data/ModelNet10/test_pc.npz")

train_data, train_labels = train_pcs['train_pc'], train_pcs["train_labels"]
test_data, test_labels = test_pcs['test_pc'], test_pcs["test_labels"]

In [5]:
# Lookup table to encode Labels to be consumed by the model
lookup = {
    "bed": [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    "bathtub": [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    "chair": [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    "desk": [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    "dresser": [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    "monitor": [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    "night_stand": [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    "sofa": [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    "table": [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    "toilet": [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
}

train_labels = np.array([lookup[i] for i in train_labels])
test_labels = np.array([lookup[i] for i in test_labels])

In [6]:
# Regualizer for the T-net training
class OrthogonalRegularizer(Regularizer):
    def __init__(self, num_features, l2reg=0.001):
        self.num_features = num_features
        self.l2reg = l2reg
        self.eye = tf.eye(num_features)

    def __call__(self, x):
        x = tf.reshape(x, (-1, self.num_features, self.num_features))
        xxt = tf.tensordot(x, x, axes=(2, 2))
        xxt = tf.reshape(xxt, (-1, self.num_features, self.num_features))
        return tf.reduce_sum(self.l2reg * tf.square(xxt - self.eye))

class PointNet:
    def __init__(self, num_points, n_classes, batch_size, epochs, tnet_enabled = False):
        self.num_points = num_points
        self.batch_size = batch_size
        self.epochs = epochs
        self.n_classes = n_classes
        self.tnet_enabled = tnet_enabled

        #### INITIALIZE THE MODEL
        self.max_pool_op = GlobalMaxPooling1D()

        # Model
        inp = Input(shape=(self.num_points, 3), name="Input")
        if self.tnet_enabled:
            x = self.__t_net(inp, 3)
            x = self.__build_pre_pool(x)
        else:
            x = self.__build_pre_pool(inp)
        x = self.max_pool_op(x)
        x = self.__build_post_pool(x)

        self.model = Model(inp, x, name="PointNet")
        self.model.compile(loss="categorical_crossentropy", optimizer = Adam(learning_rate=0.001), metrics=[CategoricalAccuracy()])
        self.model.summary()

    def predict(self, test_x):
        return self.model.predict(test_x)
    
    def __dense_layer_op(self, x, units):
        x = Dense(units)(x)
        x = BatchNormalization(momentum = 0.7)(x)
        return LeakyReLU()(x)
    
    def __t_net(self, inp, num_feats):
        # Initalise bias as the indentity matrix
        bias = Constant(np.eye(num_feats).flatten())
        reg = OrthogonalRegularizer(num_feats)

        tnet = self.__dense_layer_op(inp, 32)
        tnet = self.__dense_layer_op(tnet, 64)
        tnet = self.__dense_layer_op(tnet, 512)
        tnet = GlobalMaxPooling1D()(tnet)
        tnet = self.__dense_layer_op(tnet, 256)
        tnet = self.__dense_layer_op(tnet, 128)
        tnet = Dropout(0.6)(tnet)
        tnet = Dense(
            num_feats*num_feats,
            kernel_initializer = "zeros",
            bias_initializer = bias,
            activity_regularizer=reg,
        )(tnet)
        
        feat_trans = Reshape((num_feats,num_feats))
        op = feat_trans(tnet)
        return Dot(axes = (2, 1))([inp, op])

    def train(self):
        """
        Training Function which loads the data & trains the model
        """
        # Shuffle
        global train_data
        global train_labels
        global test_data
        global test_labels

        idcs = np.arange(len(test_data))
        np.random.shuffle(idcs)
        idcs = idcs[:int(len(test_data)*0.4)]

        train_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_labels))\
                                       .shuffle(len(train_data))\
                                       .batch(self.batch_size)
        
        test_dataset = tf.data.Dataset.from_tensor_slices((test_data[idcs], test_labels[idcs]))\
                                      .shuffle(len(idcs))\
                                      .batch(self.batch_size)
        
        self.model.fit(train_dataset, epochs=self.epochs, validation_data=test_dataset)

    def __build_pre_pool(self, x):
        """
            Helper function building the network section before pooling operation
        """
        x = self.__dense_layer_op(x, 32)
        x = self.__dense_layer_op(x, 32)
        if self.tnet_enabled:
            x = self.__t_net(x, 32)
        x = self.__dense_layer_op(x, 32)
        x = self.__dense_layer_op(x, 64)
        x = self.__dense_layer_op(x, 512)
        return x

    def __build_post_pool(self, x):
        """
            Helper function building the network section post pooling operation
        """
        x = self.__dense_layer_op(x, 256)
        x = Dropout(0.6)(x)
        x = self.__dense_layer_op(x, 128)
        x = Dropout(0.6)(x)
        x = Dense(self.n_classes, activation = 'softmax')(x)
        return x

In [7]:
pt_net = PointNet(2048, 10, BATCH_SIZE, EPOCHS, tnet_enabled = True)

2023-09-06 02:04:16.447310: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-09-06 02:04:16.447325: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "PointNet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 Input (InputLayer)          [(None, 2048, 3)]            0         []                            
                                                                                                  
 dense (Dense)               (None, 2048, 32)             128       ['Input[0][0]']               
                                                                                                  
 batch_normalization (Batch  (None, 2048, 32)             128       ['dense[0][0]']               
 Normalization)                                                                                   
                                                                                                  
 leaky_re_lu (LeakyReLU)     (None, 2048, 32)             0         ['batch_normalization[0

In [8]:
pt_net.train()

Epoch 1/6


2023-09-06 02:04:18.209821: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


333/333 [==============================] - ETA: 0s - loss: 2.5594 - categorical_accuracy: 0.5277

2023-09-06 02:05:15.504297: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


333/333 [==============================] - 60s 170ms/step - loss: 2.5594 - categorical_accuracy: 0.5277 - val_loss: 1.8238 - val_categorical_accuracy: 0.6997
Epoch 2/6
333/333 [==============================] - 54s 163ms/step - loss: 1.8525 - categorical_accuracy: 0.6794 - val_loss: 1.6496 - val_categorical_accuracy: 0.7493
Epoch 3/6
333/333 [==============================] - 54s 163ms/step - loss: 1.6422 - categorical_accuracy: 0.7339 - val_loss: 1.5527 - val_categorical_accuracy: 0.7713
Epoch 4/6
333/333 [==============================] - 54s 164ms/step - loss: 1.5415 - categorical_accuracy: 0.7595 - val_loss: 1.4330 - val_categorical_accuracy: 0.8264
Epoch 5/6
333/333 [==============================] - 55s 164ms/step - loss: 1.4143 - categorical_accuracy: 0.7830 - val_loss: 1.4101 - val_categorical_accuracy: 0.8347
Epoch 6/6
333/333 [==============================] - 54s 163ms/step - loss: 1.3538 - categorical_accuracy: 0.8047 - val_loss: 1.4058 - val_categorical_accuracy: 0.8430


## T-NET Disabled

We also have the option to disable T-net modules in our model

In [10]:
pt_net2 = PointNet(2048, 10, BATCH_SIZE, EPOCHS, tnet_enabled = False)

Model: "PointNet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 2048, 3)]         0         
                                                                 
 dense_20 (Dense)            (None, 2048, 32)          128       
                                                                 
 batch_normalization_17 (Ba  (None, 2048, 32)          128       
 tchNormalization)                                               
                                                                 
 leaky_re_lu_17 (LeakyReLU)  (None, 2048, 32)          0         
                                                                 
 dense_21 (Dense)            (None, 2048, 32)          1056      
                                                                 
 batch_normalization_18 (Ba  (None, 2048, 32)          128       
 tchNormalization)                                        

In [11]:
pt_net2.train()

Epoch 1/6


2023-09-06 02:09:50.012319: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


333/333 [==============================] - ETA: 0s - loss: 1.1221 - categorical_accuracy: 0.6470

2023-09-06 02:10:13.282001: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


333/333 [==============================] - 25s 69ms/step - loss: 1.1221 - categorical_accuracy: 0.6470 - val_loss: 0.5520 - val_categorical_accuracy: 0.8320
Epoch 2/6
333/333 [==============================] - 21s 64ms/step - loss: 0.5967 - categorical_accuracy: 0.8064 - val_loss: 0.3975 - val_categorical_accuracy: 0.8678
Epoch 3/6
333/333 [==============================] - 22s 66ms/step - loss: 0.4753 - categorical_accuracy: 0.8447 - val_loss: 0.3648 - val_categorical_accuracy: 0.8788
Epoch 4/6
333/333 [==============================] - 22s 65ms/step - loss: 0.4218 - categorical_accuracy: 0.8599 - val_loss: 0.3581 - val_categorical_accuracy: 0.8843
Epoch 5/6
333/333 [==============================] - 21s 64ms/step - loss: 0.3659 - categorical_accuracy: 0.8776 - val_loss: 0.3127 - val_categorical_accuracy: 0.8871
Epoch 6/6
333/333 [==============================] - 21s 64ms/step - loss: 0.3452 - categorical_accuracy: 0.8816 - val_loss: 0.3361 - val_categorical_accuracy: 0.8788


In [ ]:
if platform == "darwin":
    os.system("say 'Model Training DONE!'")